In [1]:
# Importing packages

import pandas as pd
from zipfile import ZipFile
import os
import math
import numpy as np
from slugify import slugify
import re

In [2]:
# Set relative filepaths
# Download link: https://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download
# Docs link: https://www.faa.gov/sites/faa.gov/files/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/ardata.pdf

__file__ = 'os.path.abspath('')'

script_dir = os.path.dirname(__file__) 
rel_path = './data/ReleasableAircraft.zip'
abs_file_path = os.path.join(script_dir, rel_path)

In [3]:
# Read zipfile

zf = ZipFile(abs_file_path) 

In [4]:
# Load the .csv file we need from the zipfile - MASTER.txt

master = pd.read_csv(zf.open('MASTER.txt'), low_memory=False)
ref = pd.read_csv(zf.open('ACFTREF.txt'))

In [5]:
# Slugify columns

master.columns = master.columns.str.replace(r'\W+', '_', regex=True)
ref.columns = ref.columns.str.replace(r'\W+', '_', regex=True)

In [6]:
# Lowercase columns

master.columns = [x.lower() for x in master.columns]
ref.columns = [x.lower() for x in ref.columns]

In [7]:
master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288062 entries, 0 to 288061
Data columns (total 35 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   n_number          288062 non-null  object 
 1   serial_number     288062 non-null  object 
 2   mfr_mdl_code      288062 non-null  object 
 3   eng_mfr_mdl       288062 non-null  object 
 4   year_mfr          288062 non-null  object 
 5   type_registrant   288062 non-null  object 
 6   name              288062 non-null  object 
 7   street            288062 non-null  object 
 8   street2           288062 non-null  object 
 9   city              288062 non-null  object 
 10  state             288062 non-null  object 
 11  zip_code          288062 non-null  object 
 12  region            288062 non-null  object 
 13  county            288062 non-null  object 
 14  country           288062 non-null  object 
 15  last_action_date  288062 non-null  int64  
 16  cert_issue_date   28

In [8]:
# Strip leading and trailing underscores on some column names

master.columns = master.columns.str.rstrip('_')
master.columns = master.columns.str.lstrip('_')

In [9]:
# Drop the empty columns at the end of each file (the 'unnamed: xx' fields)

master.drop('unnamed_34', inplace=True, axis=1)
ref.drop('unnamed_13', inplace=True, axis=1)

In [10]:
master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288062 entries, 0 to 288061
Data columns (total 34 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   n_number          288062 non-null  object
 1   serial_number     288062 non-null  object
 2   mfr_mdl_code      288062 non-null  object
 3   eng_mfr_mdl       288062 non-null  object
 4   year_mfr          288062 non-null  object
 5   type_registrant   288062 non-null  object
 6   name              288062 non-null  object
 7   street            288062 non-null  object
 8   street2           288062 non-null  object
 9   city              288062 non-null  object
 10  state             288062 non-null  object
 11  zip_code          288062 non-null  object
 12  region            288062 non-null  object
 13  county            288062 non-null  object
 14  country           288062 non-null  object
 15  last_action_date  288062 non-null  int64 
 16  cert_issue_date   288062 non-null  obj

In [11]:
ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90307 entries, 0 to 90306
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   code            90307 non-null  object
 1   mfr             90307 non-null  object
 2   model           90307 non-null  object
 3   type_acft       90307 non-null  object
 4   type_eng        90307 non-null  int64 
 5   ac_cat          90307 non-null  int64 
 6   build_cert_ind  90307 non-null  int64 
 7   no_eng          90307 non-null  int64 
 8   no_seats        90307 non-null  int64 
 9   ac_weight       90307 non-null  object
 10  speed           90307 non-null  int64 
 11  tc_data_sheet   90307 non-null  object
 12  tc_data_holder  90307 non-null  object
dtypes: int64(6), object(7)
memory usage: 9.0+ MB


In [12]:
# Rename the code col in the ref dataframe

ref.rename(columns = {'code':'mfr_mdl_code'}, inplace = True)

In [13]:
# Merging the two dataframes together on the common field - mfr_mdl_code in master and code in ref

joined = pd.merge(ref, master, on='mfr_mdl_code', how='inner')

# joined = pd.concat([master.set_index('mfr_mdl_code'),ref.set_index('code')], axis=1, join='inner')

# joined = pd.concat([master, ref], axis=0)

In [14]:
# How many records are in the database?
# Are any records missing?

joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288062 entries, 0 to 288061
Data columns (total 46 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   mfr_mdl_code      288062 non-null  object
 1   mfr               288062 non-null  object
 2   model             288062 non-null  object
 3   type_acft         288062 non-null  object
 4   type_eng          288062 non-null  int64 
 5   ac_cat            288062 non-null  int64 
 6   build_cert_ind    288062 non-null  int64 
 7   no_eng            288062 non-null  int64 
 8   no_seats          288062 non-null  int64 
 9   ac_weight         288062 non-null  object
 10  speed             288062 non-null  int64 
 11  tc_data_sheet     288062 non-null  object
 12  tc_data_holder    288062 non-null  object
 13  n_number          288062 non-null  object
 14  serial_number     288062 non-null  object
 15  eng_mfr_mdl       288062 non-null  object
 16  year_mfr          288062 non-null  obj

In [15]:
# Are there duplicates?
# Putting all of the duplicate rows into a dataframe of their own

joined2 = joined[joined.duplicated()]

In [16]:
# There are not any duplicates.

joined2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   mfr_mdl_code      0 non-null      object
 1   mfr               0 non-null      object
 2   model             0 non-null      object
 3   type_acft         0 non-null      object
 4   type_eng          0 non-null      int64 
 5   ac_cat            0 non-null      int64 
 6   build_cert_ind    0 non-null      int64 
 7   no_eng            0 non-null      int64 
 8   no_seats          0 non-null      int64 
 9   ac_weight         0 non-null      object
 10  speed             0 non-null      int64 
 11  tc_data_sheet     0 non-null      object
 12  tc_data_holder    0 non-null      object
 13  n_number          0 non-null      object
 14  serial_number     0 non-null      object
 15  eng_mfr_mdl       0 non-null      object
 16  year_mfr          0 non-null      object
 17  type_registrant   0 non-null

In [17]:
# To test the values, we need just the name values without the extra whitespace

joined.name = joined.name.map(str.strip)

In [18]:
# There are a variety of name lengths incl as of 1/2023 update 487 records with no name

joined['name_len'] = joined['name'].astype(str).map(len)
joined.name_len.value_counts()

15    29346
16    28797
14    27579
13    23782
17    21107
20    18316
18    17527
19    16364
12    14428
21    10902
11     9022
22     7914
24     6554
23     6468
10     6271
25     5490
26     4525
27     4503
29     3347
30     3047
9      2863
28     2853
31     2454
35     2046
8      1688
32     1388
7      1223
33     1200
41     1047
34      940
38      865
36      659
40      590
37      531
39      492
0       487
45      260
43      204
42      194
44      160
46      128
48      107
47      100
50       88
49       87
6        77
4        16
5        14
3        11
2         1
Name: name_len, dtype: int64

In [19]:
# We need the following fields to concatenate for the "registrant" field in the publicaccountability.org search:
# name, street, street2, city, state
# So let's make sure those are solid
# We also need the cert_issue_date for the date field, the manufacturer field, and to add a year field (2023)

In [20]:
# All of the fields appear to have the same num of chars, use .strip()

joined.street = joined.street.map(str.strip)

In [21]:
# There are a variety of address lengths incl as of the 1/2023 update 490 that are blank
# Should not be longer than 33 chars

joined['street_len'] = joined['street'].astype(str).map(len)
joined.street_len.value_counts()

15    28268
16    26856
17    25340
14    23446
18    21606
13    20019
19    17980
10    16393
20    14073
12    14017
11    13625
21    13191
22     8900
23     7723
26     6887
24     6367
25     5197
27     4498
9      3423
28     2744
30     1516
29     1445
31     1182
32      950
8       873
33      733
0       490
7       185
4        72
6        42
5        14
3         7
Name: street_len, dtype: int64

In [22]:
# All of the fields appear to have the same num of chars, use .strip()

joined.street2 = joined.street2.map(str.strip)

In [23]:
# There are a variety of name lengths incl as of 1/2023 update 277519 with no values in this field
# Should not be longer than 

joined['street2_len'] = joined['street2'].astype(str).map(len)
joined.street2_len.value_counts()

0     277519
16      1206
24       893
18       876
8        869
15       774
31       764
11       622
17       613
13       391
19       386
14       327
21       318
26       311
23       291
20       290
12       285
22       216
7        197
10       185
28       120
25       119
27       114
9         92
30        85
33        48
29        40
32        34
5         29
6         27
4         13
2          6
3          2
Name: street2_len, dtype: int64

In [24]:
# All of the fields appear to have the same num of chars, use .strip()

joined.city = joined.city.map(str.strip)

In [25]:
# There are a variety of name lengths incl as of 1/2023 update 277519 with no values in this field
# Should not be longer than 18 chars

joined['city_len'] = joined['city'].astype(str).map(len)
joined.city_len.value_counts()

7     42334
10    40441
6     38281
9     37614
8     34818
11    23694
5     17477
12    14942
13    11137
14     8737
4      7021
15     4661
16     3861
17     1395
18      635
3       536
0       477
2         1
Name: city_len, dtype: int64

In [26]:
# All of the fields appear to have the same num of chars, use .strip()

joined.state = joined.state.map(str.strip)

In [27]:
# There are a variety of name lengths incl as of 1/2023 update 1936 with no values in this field
# Should not be longer than 2 chars

joined['state_len'] = joined['state'].astype(str).map(len)
joined.state_len.value_counts()

2    286126
0      1936
Name: state_len, dtype: int64

In [28]:
# All of the fields appear to have the same num of chars, use .strip()

joined.mfr = joined.mfr.map(str.strip)

In [29]:
# There are a variety of mfr lengths
# Should not be longer than 30 chars

joined['mfr_len'] = joined['mfr'].astype(str).map(len)
joined.mfr_len.value_counts()

6     88839
5     64964
18    12479
14    11101
7     10462
8      8733
15     7954
11     6879
20     6805
17     6758
16     5983
13     5559
4      5021
27     4472
19     4446
22     4344
10     4273
12     4027
25     3952
30     3278
9      3160
21     2932
26     2852
24     2620
23     2296
3      2082
29     1152
28      638
2         1
Name: mfr_len, dtype: int64

In [30]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288062 entries, 0 to 288061
Data columns (total 52 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   mfr_mdl_code      288062 non-null  object
 1   mfr               288062 non-null  object
 2   model             288062 non-null  object
 3   type_acft         288062 non-null  object
 4   type_eng          288062 non-null  int64 
 5   ac_cat            288062 non-null  int64 
 6   build_cert_ind    288062 non-null  int64 
 7   no_eng            288062 non-null  int64 
 8   no_seats          288062 non-null  int64 
 9   ac_weight         288062 non-null  object
 10  speed             288062 non-null  int64 
 11  tc_data_sheet     288062 non-null  object
 12  tc_data_holder    288062 non-null  object
 13  n_number          288062 non-null  object
 14  serial_number     288062 non-null  object
 15  eng_mfr_mdl       288062 non-null  object
 16  year_mfr          288062 non-null  obj

In [31]:
joined.drop(['name_len', 'street_len', 'street2_len', 'city_len', 'state_len', 'mfr_len'], axis=1, inplace=True)

In [32]:
# Creating a US 5-digit ZIP code field
# This will fill the uszip5 field with ZEROES where the country is not USA

joined['uszip5'] = np.where(joined['country'] == 'US', joined['zip_code'].astype(str).str[:5], '')

In [35]:
# This is to be sure the 5-digit ZIPs that begin with leading 0s do have the leading 0s -
# Python strips them, and, when we export to .csv, they will not show up in Excel - so the ZIP 01234 would appear as 1234
# However, if you open the file in Sublime Text, the 0s are there

joined['uszip5'] = joined['uszip5'].str.zfill(5)

In [33]:
# Add year field

joined['year'] = 2023

In [36]:
# Export
# Per the note here: https://registry.faa.gov/aircraftinquiry:
# The duration of aircraft registration certificates has been extended up to 7 years. 
# The Registry will be issuing revised certificates in batches based on the former expiration date. 
# For verification purposes, even though the expiration date on the registration certificate may not match the expiration 
# date in the FAA Aircraft Registration database, any registration certificate displaying an expiration date of January 31, 2023 
# or later is still valid. This applies to all foreign Civil Aviation Authorities or anyone else with a verification need.

joined.to_csv('aircraft_2023.csv', index=False)

In [ ]:
# Here's a bunch of code I wrote before I realized we didn't need it 

# Check numeric date ranges. Anything too high or too low?
# There are a lot of numerical fields in this dataset. We'll check the ones with numbers only - 
# There are some fields with a combination of letters and numbers
# Fields with numerical values to check:
# type_eng, ac_cat, build_cert_ind, no_eng, no_seats, speed, last_action_date, type_engine, mode_s_code 
# Similar to what I did with the pilots data, I'll use value_counts

In [18]:
# The type_eng values should be between or equal to 1 and 11, per docs

joined.type_eng.value_counts()

1     203530
5      25427
8      19159
2      12889
0       8723
3       8446
7       4711
10      3343
4       1751
11        73
9          5
6          5
Name: type_eng, dtype: int64

In [19]:
# The ac_cat values should be 1, 2 or 3, per docs

joined.ac_cat.value_counts()

1    285018
3      2837
2       207
Name: ac_cat, dtype: int64

In [20]:
# The build_cert_ind values should be 1, 2 or 3, per docs

joined.build_cert_ind.value_counts()

0    235981
1     43065
2      9016
Name: build_cert_ind, dtype: int64

In [21]:
# The no_eng values can vary. This field is the number of engines in the registered aircraft.

joined.no_eng.value_counts()

1     226124
2      48589
0       8730
4       2466
3       1117
6        650
8        266
5         61
9         14
12        13
10        13
20         5
13         5
16         4
14         1
32         1
18         1
7          1
33         1
Name: no_eng, dtype: int64

In [22]:
# The no_seats values can vary. This field is the number of seats in the registered aircraft.

joined.no_seats.value_counts()

4      96271
2      82537
6      27454
1      16979
5       9168
       ...  
131        1
128        1
250        1
273        1
195        1
Name: no_seats, Length: 171, dtype: int64

In [23]:
# The speed values can vary. This field is the registered aircraft's cruising speed.
# As of the January update, one plane is listed with a cruising speed of 1,125
# Upon further inspection, the plane is a single-engine and weighs less than 12,500 pounds, so I think this is an error.

joined.speed.value_counts()

0       135675
105      21905
90       15269
107      10760
108       8395
         ...  
177          1
282          1
62           1
237          1
1125         1
Name: speed, Length: 201, dtype: int64

In [24]:
# The last_action_date values will be dates formatted as YYYYMMDD
# To really investigate the integrity of this field, we would need to look separately at YYYY, MM, DD
# But we'll just leave that instruction here for anyone working with this data, and for this script
# I will just make sure that the values are the proper length
# To do that we will need to convert it to a string first

joined['last_action_date'] = joined['last_action_date'].astype(str)

In [25]:
# Now check if it's too short or too long
# Create a new column of lengths for each value in last_action_date

joined['last_action_datelen'] = joined['last_action_date'].astype(str).map(len)

In [26]:
# Check values in created col
# Should be 8

joined.last_action_datelen.value_counts()

8    288062
Name: last_action_datelen, dtype: int64

In [27]:
# Type engine should be int values 0 through 11

joined.type_engine.value_counts()

1     203506
5      25425
8      19175
2      12891
0       8723
3       8448
7       4711
10      3346
4       1754
11        73
9          5
6          5
Name: type_engine, dtype: int64

In [28]:
# The values of mode_s_code vary but should be 8 characters in length 
# Change to string first

joined['mode_s_code'] = joined['mode_s_code'].astype(str)

In [29]:
# Now check if it's too short or too long
# Create a new column of lengths for each value

joined['mode_s_codelen'] = joined['mode_s_code'].astype(str).map(len)

In [30]:
# Check values in created col
# Should be 8

joined.mode_s_codelen.value_counts()

8    288062
Name: mode_s_codelen, dtype: int64

In [31]:
# Now drop the len cols created 

joined = joined.drop(['last_action_datelen', 'mode_s_codelen'], axis=1)

In [32]:
# Is there anything blank or missing?

joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288062 entries, 0 to 288061
Data columns (total 46 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   mfr_mdl_code      288062 non-null  object
 1   mfr               288062 non-null  object
 2   model             288062 non-null  object
 3   type_acft         288062 non-null  object
 4   type_eng          288062 non-null  int64 
 5   ac_cat            288062 non-null  int64 
 6   build_cert_ind    288062 non-null  int64 
 7   no_eng            288062 non-null  int64 
 8   no_seats          288062 non-null  int64 
 9   ac_weight         288062 non-null  object
 10  speed             288062 non-null  int64 
 11  tc_data_sheet     288062 non-null  object
 12  tc_data_holder    288062 non-null  object
 13  n_number          288062 non-null  object
 14  serial_number     288062 non-null  object
 15  eng_mfr_mdl       288062 non-null  object
 16  year_mfr          288062 non-null  obj

In [33]:
# Is there information in the wrong field?
# Let's look at all of the fields and what's supposed to be in them:
# The following fields are in the documents downloaded from the FAA website but they do not appear in the files as of Jan. 2023
# Aircraft Mfr Model Code - this is missing from the MASTER.txt file
# Now we'll actually check what values should be in each field and if the values in the data match the prescribed values
# It is worth noting that all fields appear to be all the same length

In [34]:
# The n-number field should not be longer than 5 chars
# Create a new column for any values that are longer than 5 chars (we will delete this col later)
# Print it, and if there are 0 rows, we're good to go

mask = ((joined['n_number'].str.len()) > 5)
test_df = joined.loc[mask]
print(test_df)

Empty DataFrame
Columns: [mfr_mdl_code, mfr, model, type_acft, type_eng, ac_cat, build_cert_ind, no_eng, no_seats, ac_weight, speed, tc_data_sheet, tc_data_holder, n_number, serial_number, eng_mfr_mdl, year_mfr, type_registrant, name, street, street2, city, state, zip_code, region, county, country, last_action_date, cert_issue_date, certification, type_aircraft, type_engine, status_code, mode_s_code, fract_owner, air_worth_date, other_names_1, other_names_2, other_names_3, other_names_4, other_names_5, expiration_date, unique_id, kit_mfr, kit_model, mode_s_code_hex]
Index: []

[0 rows x 46 columns]


In [35]:
# Serial numbers can be as short as three characters or as long as 30

mask = ((joined['serial_number'].str.len()) > 30)
test_df = joined.loc[mask]
print(test_df)

Empty DataFrame
Columns: [mfr_mdl_code, mfr, model, type_acft, type_eng, ac_cat, build_cert_ind, no_eng, no_seats, ac_weight, speed, tc_data_sheet, tc_data_holder, n_number, serial_number, eng_mfr_mdl, year_mfr, type_registrant, name, street, street2, city, state, zip_code, region, county, country, last_action_date, cert_issue_date, certification, type_aircraft, type_engine, status_code, mode_s_code, fract_owner, air_worth_date, other_names_1, other_names_2, other_names_3, other_names_4, other_names_5, expiration_date, unique_id, kit_mfr, kit_model, mode_s_code_hex]
Index: []

[0 rows x 46 columns]


In [36]:
# Engine manufacturer mode code should be 5 chars 

mask = ((joined['eng_mfr_mdl'].str.len()) != 5)
test_df = joined.loc[mask]
print(test_df)

Empty DataFrame
Columns: [mfr_mdl_code, mfr, model, type_acft, type_eng, ac_cat, build_cert_ind, no_eng, no_seats, ac_weight, speed, tc_data_sheet, tc_data_holder, n_number, serial_number, eng_mfr_mdl, year_mfr, type_registrant, name, street, street2, city, state, zip_code, region, county, country, last_action_date, cert_issue_date, certification, type_aircraft, type_engine, status_code, mode_s_code, fract_owner, air_worth_date, other_names_1, other_names_2, other_names_3, other_names_4, other_names_5, expiration_date, unique_id, kit_mfr, kit_model, mode_s_code_hex]
Index: []

[0 rows x 46 columns]


In [37]:
# Year should be 4 chars

mask = ((joined['year_mfr'].str.len()) != 4)
test_df = joined.loc[mask]
print(test_df)

Empty DataFrame
Columns: [mfr_mdl_code, mfr, model, type_acft, type_eng, ac_cat, build_cert_ind, no_eng, no_seats, ac_weight, speed, tc_data_sheet, tc_data_holder, n_number, serial_number, eng_mfr_mdl, year_mfr, type_registrant, name, street, street2, city, state, zip_code, region, county, country, last_action_date, cert_issue_date, certification, type_aircraft, type_engine, status_code, mode_s_code, fract_owner, air_worth_date, other_names_1, other_names_2, other_names_3, other_names_4, other_names_5, expiration_date, unique_id, kit_mfr, kit_model, mode_s_code_hex]
Index: []

[0 rows x 46 columns]


In [38]:
# Type registrant can be any value from 1 to 9

mask = ((joined['type_registrant'].str.len()) < 1 & (joined['type_registrant'].str.len() > 9))
test_df = joined.loc[mask]
print(test_df)

Empty DataFrame
Columns: [mfr_mdl_code, mfr, model, type_acft, type_eng, ac_cat, build_cert_ind, no_eng, no_seats, ac_weight, speed, tc_data_sheet, tc_data_holder, n_number, serial_number, eng_mfr_mdl, year_mfr, type_registrant, name, street, street2, city, state, zip_code, region, county, country, last_action_date, cert_issue_date, certification, type_aircraft, type_engine, status_code, mode_s_code, fract_owner, air_worth_date, other_names_1, other_names_2, other_names_3, other_names_4, other_names_5, expiration_date, unique_id, kit_mfr, kit_model, mode_s_code_hex]
Index: []

[0 rows x 46 columns]


In [39]:
# The next few fields are long-text fields; registrant's name, address (street and street 2), city, state
# This line checks to make sure none of these fields have numbers
# There are no numbers

x = joined.name.str.isalpha()
print(x.value_counts())

False    288062
Name: name, dtype: int64
